In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
train = pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
train.head()

,Id,Location,Weekly Cases,Year,Weekly Cases per Million,Weekly Deaths,Weekly Deaths per Million,Total Vaccinations,People Vaccinated,People Fully Vaccinated,Total Boosters,Daily Vaccinations,Total Vaccinations per Hundred,People Vaccinated per Hundred,People Fully Vaccinated per Hundred,Total Boosters per Hundred,Daily Vaccinations per Hundred,Daily People Vaccinated,Daily People Vaccinated per Hundred,Next Week's Deaths
0,772016845,Costa Rica,15436.0,2021,2994.980,122.0,23.671,950252.0,605099.0,345153.0,NaN,18767.0,18.44,11.74,6.70,NaN,3641.0,9488.0,0.184,140.0
1,537953416,Dominican Republic,3991.0,2021,358.972,61.0,5.487,105000.0,105000.0,NaN,NaN,11806.0,0.94,0.94,NaN,NaN,1062.0,11806.0,0.106,73.0
2,363835805,Brazil,456899.0,2022,2131.792,4187.0,19.536,392925626.0,177836868.0,155071690.0,65073212.0,814209.0,183.33,82.97,72.35,30.36,3799.0,173043.0,0.081,3212.0
3,677248247,Lesotho,17.0,2022,7.451,0.0,0.000,NaN,NaN,NaN,NaN,1796.0,NaN,NaN,NaN,NaN,787.0,1399.0,0.061,0.0
4,493133731,Panama,5591.0,2020,1284.913,110.0,25.280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0


In [10]:
df_all=pd.concat([train,test],axis=0)

In [11]:
df_all=df_all.fillna(0)
df_all["Year"]=df_all["Year"].map({2020:0,2021:1,2022:2})
df_all=df_all.drop(["Id","Location","Year","Total Vaccinations per Hundred","People Vaccinated per Hundred",\
                            "People Vaccinated per Hundred","People Fully Vaccinated per Hundred",\
                            "Total Boosters per Hundred","Daily Vaccinations per Hundred","Weekly Cases per Million"],axis=1)

In [12]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129156 entries, 0 to 29155
Data columns (total 11 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   Weekly Cases                         129156 non-null  float64
 1   Weekly Deaths                        129156 non-null  float64
 2   Weekly Deaths per Million            129156 non-null  float64
 3   Total Vaccinations                   129156 non-null  float64
 4   People Vaccinated                    129156 non-null  float64
 5   People Fully Vaccinated              129156 non-null  float64
 6   Total Boosters                       129156 non-null  float64
 7   Daily Vaccinations                   129156 non-null  float64
 8   Daily People Vaccinated              129156 non-null  float64
 9   Daily People Vaccinated per Hundred  129156 non-null  float64
 10  Next Week's Deaths                   129156 non-null  float64
dtypes: float64(11)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
Y=df_all["Next Week's Deaths"]
df_all=df_all.drop(["Next Week's Deaths"],axis=1)
df_all=scaler.fit_transform(df_all)
X=df_all[:train.shape[0]]

X_train,X_test,Y_train,Y_test=train_test_split(X,Y[:train.shape[0]],test_size=0.2,random_state=33)

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rf=RandomForestRegressor(n_estimators=200,max_features="sqrt",oob_score=True)
rf.fit(X_train,Y_train)
pred=rf.predict(X_test)
print('RMSE: ', np.sqrt(mean_squared_error(Y_test, pred)))

RMSE:  325.86668473779764


In [16]:
ans=rf.predict(df_all[train.shape[0]:])
ans

array([956.82545414, 793.79045414, 778.40545414, ...,  12.125     ,
        12.25      ,   9.705     ])

In [17]:
sub = pd.DataFrame()
sub['Id'] = test["Id"]
sub["Next Week's Deaths"] = ans
sub.to_csv('submission.csv',index=False)